# Make Classifier From Scratch
최소한의 패키지를 가지고 다중 클래스분류기 만들기 프로젝트

2018310412 인공지능융합전공 고준서

### 전처리기 만들기
clean_text( ) 함수에 도큐먼트 묶음을 넣으면 전처리 한 후 다시 돌려보냄

In [4]:
from nltk.corpus import names
all_names = set(names.words())

/Users/gojunseo/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
def letters_only(word):
    return word.isalpha()

In [7]:
def clean_text(doc):
    cleaned_doc = []
    for word in doc.split(' '):
        word = word.lower()
  
        if letters_only(word) and word not in all_names and len(word) > 2: # remove number and punc. and name entity
            cleaned_doc.append(lemmatizer.lemmatize(word, "v"))
    return ' '.join(cleaned_doc) 

### NaiveBayes Classifier를 만들어준다.
class로 만들어서 여러번의 함수를 호출하지 않고도 동일한 작업을 수행할 수 있도록 한다.
smoothing옵션을 클래스의 초기 파라미터로 줄 수 있다. 기본값은 0이다. 그러므로 0을 방지하려면 1로 스무딩 값을 주어야 한다.

In [181]:
from collections import defaultdict
import numpy as np
class NaiveBayes:
    def __init__(self, smoothing = 0):
        self.smoothing = smoothing
        self.term_docs_train_dict = {}
        self.y_ = {}
        
    def _split_sort(self, X, y):
#         print(X.shape)
        for label in range(self.how_many_label):
            y_temp = []
            x_temp = []
            for i in range(len(y)):
#                 print(i)
                if y[i] == label:
                    y_temp.append(1)
                    x_temp.append(X[i])
                else:
                    y_temp.append(-1)
                    x_temp.append(X[i])
            self.term_docs_train_dict[label] = np.array(x_temp)
            self.y_[label] = y_temp
    
    def _get_label_index(self, labels):
        label_index = defaultdict(list)
        for index, label in enumerate(labels):
            label_index[label].append(index)
        return label_index
    
    def _get_prior(self, label_index):
        prior = {label: len(index) for label, index in label_index.items()}
        pr_zero = prior[1]
        pr_one = prior[-1]

        prior[1] = pr_zero/(pr_zero+ pr_one)
        prior[-1] = pr_one/(pr_zero+pr_one)

        return prior
    
    def _get_likelihood(self, term_doc_matrix, label_index, smoothing=0):
        likelihood = {}
        word_count = term_doc_matrix.shape[1]
        total_word_len_in_s = {}
        total_word_set_in_s = {}
        for label, index in label_index.items():
            whole = 0
            for ix in index:
                whole += sum(term_doc_matrix[ix])
            total_word_len_in_s[label] = whole
        for label, index in label_index.items():
            whole = 0
            for ix in index:
                count = 0
                for tmp in term_doc_matrix[ix]:
                    if tmp != 0:
                        count += 1
                whole += count
            total_word_set_in_s[label] = whole
        
        likelihood[1] = np.zeros(word_count)
        likelihood[-1] = np.zeros(word_count)
        
        lab_one = np.array([term_doc_matrix[idx] for idx in label_index[-1]])
        lab_zero = np.array([term_doc_matrix[idx] for idx in label_index[1]])
        print("lab",lab_one, lab_zero)
        sum_one = lab_one.sum(axis = 0)
        sum_zero = lab_zeroasum(axis = 0)
        print("sum", sum_one.shape, sum_zero)
        for i in range(word_count):
            likelihood[1][i] = (sum_zero[i]+smoothing) / (total_word_len_in_s[1]+total_word_set_in_s[1])
            likelihood[-1][i] = (sum_one[i]+smoothing) / (total_word_len_in_s[-1]+total_word_set_in_s[-1])
        
        return likelihood
    
    
    def _get_posterior(self, term_doc_matrix, prior, likelihood):

        a = []
        tt = term_doc_matrix.sum(axis=1)
        for i in range(term_doc_matrix.shape[0]):
            a.append((term_doc_matrix[i]/tt[i])*100)
        term_doc_matrix = np.array(a)

        num_docs = term_doc_matrix.shape[0]
        posteriors = [None for i in range(num_docs)]
        posterior_yes = likelihood[1]
        posterior_no = likelihood[-1]

#         print("posteriro", posterior_yes, posterior_no)

        posterior_yes = (np.log(posterior_yes) * term_doc_matrix).sum(axis = 1) + np.log(prior[1])
        posterior_no = (np.log(posterior_no) * term_doc_matrix).sum(axis = 1) + np.log(prior[-1])
        posterior_yes = np.exp(posterior_yes)
        posterior_no = np.exp(posterior_no)
        num_docs = term_doc_matrix.shape[0]

        for i in range(num_docs):
            if posterior_yes[i] == 0 and posterior_no[i] == 0:
                if posterior_yes[i] < posterior_no[i]:        
                    tmp = {-1: 0, 1:1}
                else:
                    tmp = {-1:1,1:0}
                posteriors[i] =tmp
                continue

            tmp = {-1: posterior_no[i]/(posterior_yes[i]+posterior_no[i]), 1:posterior_yes[i]/(posterior_yes[i]+posterior_no[i])}

            posteriors[i] =tmp

        return posteriors

    
    def fit(self, X, y):
        X = X.toarray()
        y = list(y)
        self.how_many_label = len(set(y))
        self._split_sort(X, y)
        self.label_index_dict = {}
        for lab in range(self.how_many_label):
            label_index = self._get_label_index(self.y_[lab])
            self.label_index_dict[lab] = label_index
        print("label_index", self.label_index_dict)
        self.prior_dict = {}
        for lab in range(self.how_many_label):
            prior = self._get_prior(self.label_index_dict[lab])
            self.prior_dict[lab] = prior
        print("prior_dict",self.prior_dict)
        self.likelihood_dict = {}
        for lab in range(self.how_many_label):
            likelihood = self._get_likelihood(self.term_docs_train_dict[lab], self.label_index_dict[lab], self.smoothing) 
            self.likelihood_dict[lab] = likelihood
        print("self.likelihood_dict", self.likelihood_dict)
    def predict(self, X):
        X = X.toarray()
        posteriors_dict = {}
        for lab in range(self.how_many_label):
            posteriors = self._get_posterior(X, self.prior_dict[lab], self.likelihood_dict[lab])
            posteriors_dict[lab] = posteriors
#         print("postdic",posteriors_dict)
        a = {}
        for text in range(X.shape[0]):
            max_id = -1
            max_val = -float("inf")
            for lab in range(5):    
                if max_val < posteriors_dict[lab][text][1]:
                    max_id = lab
                    max_val = posteriors_dict[lab][text][1]
            a[text] = max_id
        return np.array(list(a.values()))

### 정확도 측정 함수
a,b의 값을 비교해 같은 것의 개수를 센다. 그 후 전체 개수로 나눈다.

In [145]:
def accuracy_function(a, b):
    b = list(b)
    a = list(a)
    count = 0
    for i in range(len(b)):
        if a[i] == b[i]:
            count+= 1
    return count / len(b)

### pandas를 통해 train과 test에 해당하는 csv파일을 불러와 저장한다.

In [146]:
import pandas as pd
train_df = pd.read_csv("TRAIN.csv")
test_df = pd.read_csv("TEST.csv")
x_train = train_df.iloc[:, 2]
y_train = train_df.iloc[:,-1].astype(int)
x_train = [clean_text(doc) for doc in x_train]
x_test = test_df["Post"]
x_test = [clean_text(doc) for doc in x_test]
print(len(x_train), len(x_test))

350 150


In [147]:
from sklearn.model_selection import train_test_split
train_x_train,test_x_train, train_y_train, test_y_train = train_test_split(x_train, y_train, test_size = 0.1)

In [148]:
len(train_x_train), len(test_x_train), len(train_y_train), len(test_y_train)

(315, 35, 315, 35)

### 라벨이 있는 train data를 분리한 것을 통해 정확도를 테스트한다.
제작한 Vectorizer, NaiveBayes를 활용하여 분석한다.

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

NB = NaiveBayes(smoothing = 0.1)
pipeline = Pipeline([
    ("Vec", TfidfVectorizer()),
    ("NB", NaiveBayes()),
])
parameters_pipeline = {
    "Vec__max_df":(0.1,0.5, 1.0, 10),
    "Vec__ngram_range":[(2,2), (2,1), (1,1)],
    "Vec__min_df":[1,5,10,20],
    "Vec__smooth_idf":[True, False],
    "NB__smoothing":[1,0.5,0.1,0.01]
}

In [85]:
grid_search = GridSearchCV(pipeline, parameters_pipeline, n_jobs=-1, cv = 3)
grid_search.fit(train_x_train, train_y_train)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(memory=None,
         steps=[('Vec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('NB', <__main__.NaiveBayes object at 0x13040f410>)],

In [197]:
term_docs_test[20].toarray()

array([[ 1,  4,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         3,  0,  0,  0,  0,  5,  0,  0,  0,  1,  2,  0,  0,  0,  3,  0,
         0,  0,  0,  1,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,
         1,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  1,
         0,  1,  0,  1,  0,  0,  1,  0,  3,  0,  0,  0,  1,  1,  0,  0,
         1,  1,  0,  0,  0, 10,  0,  0,  3,  0,  3,  4,  0,  2,  1,  0,
         3,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  1,  1,  0,
         0,  7,  0,  2,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  7,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  5,  3,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  1,  0,  0,  0,  0,  4,
         1,  0,  0,  1,  5,  0,  0,  0,  0,  0,  0,  1,  0,  1,  1,  0,
         0,  2,  0,  0,  0,  0,  0,  0,  5,  0,  0,  1,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0,  0,  6,  2,  0,  0,  3,  0,  0,  1, 

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features= 1000, smooth_idf= False, max_df=500, ngram_range = (0,2))
cv = CountVectorizer(max_features=1000, max_df=500, stop_words="english")
cv.fit(train_x_train + test_x_train)
term_docs_train = cv.transform(train_x_train)
term_docs_test = cv.transform(test_x_train)

In [198]:
NB = NaiveBayes(smoothing = 0.1)
NB.fit(term_docs_train, train_y_train)
y_pred = NB.predict(term_docs_test)

label_index {0: defaultdict(<class 'list'>, {-1: [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 23

lab [[ 0  0  0 ...  0  0  0]
 [ 1  2  0 ...  0  1  0]
 [ 0  7  0 ...  0 16  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 2 0 ... 0 2 4]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 5 1]]
sum (1000,) [  3  40   3  15  17   2  13   3   7  14   5  19   9   4   0   4  55   9
   3   6   6  12  15   6  15  12  13  11  10  11  13  16   8   9   9   9
  27   2  11   7   8   2   1   3   5   3  19   2   2  31   7   5   2   1
   3  60   3   5  16  13  20  15   1   4   2   4   5  47   8   8   0   0
  60   8   4   6   5  15   2   6   4   4   3  11  13 141  56   2  47   4
  74  16   4  13   8   0  28   3   4   5   2   5   7   8   5  10   4   2
  10  17  34   5   1  33   5   7   2   2   6   5   5   4  17   6   5  12
  76   2   7   4   0   1  31   4   9   6   2  11  66   2   4   1   7   5
  12   5   7   9  16   2   3   4  11   4   6   2  18   3   2   7  91   4
   4  10  19   5   3   3   4   5  14   7   5   1   

lab [[ 0  0  0 ...  0  0  0]
 [ 1  2  0 ...  0  1  0]
 [ 0  7  0 ...  0 16  0]
 ...
 [ 0  3  0 ...  0 21  8]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 6 0]
 ...
 [0 0 0 ... 0 6 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
sum (1000,) [ 16  83  14  16  14   5  15   3   0  13   6  26  14   3   3   5  40  15
   3   7   4  35   8   7  10   8  16   5  17   3  14  16   8  12   9  22
  32   2   6   4   2   2   4   6  17   1  14   1   8  41   5   5   9   7
   4  63   8  13   8   4  15   9   2   7   2  10   4  46  10   8   4   8
  64   2   2   7   7   9   5   5  10  11   7  13  19 264  88   7  85   6
 143  39   4   1   5   3  51   5  14   1   8   5  16  13   5  11   2   8
  11   9  38   3   2  35  15  15   3   2   8   7   4   9   6   5   4   8
 118   6   9  21   4   8  27   1   5   9   7  31  76   5  12   5  20   6
  19   7   2  10  13   6   3   4  14   5  14  37  20   3   3  31 107  12
  12  10  18  11  15   3   5  11  25   3   7   6   

In [199]:
accuracy_function(y_pred, test_y_train)

0.2857142857142857

In [200]:
y_pred, test_y_train

(array([4, 4, 3, 0, 1, 2, 3, 4, 4, 1, 4, 4, 1, 3, 3, 3, 3, 2, 3, 4, 4, 4,
        4, 4, 1, 4, 4, 4, 4, 4, 4, 0, 1, 3, 2]),
 323    3
 227    4
 103    4
 275    4
 214    2
 254    2
 245    2
 26     2
 125    4
 277    1
 233    3
 288    2
 335    0
 57     3
 18     2
 171    3
 187    4
 347    2
 34     4
 339    3
 271    3
 331    3
 126    2
 273    4
 2      2
 22     4
 213    0
 37     1
 169    4
 349    2
 216    1
 32     4
 123    2
 161    1
 184    3
 Name: LabelNumber, dtype: int64)

### 라벨이 없는 test data를 예측하여 csv파일로 바꾸어준다.


In [205]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features= 1000, smooth_idf= False, max_df=500, ngram_range = (0,2))
cv = CountVectorizer(max_features=1000, max_df=500, stop_words="english", ngram_range=(0,2))
cv.fit(x_train)
term_docs_train = cv.transform(x_train)
test_term_docs = cv.transform(x_test)

In [206]:
NB = NaiveBayes(smoothing = 0.1)
NB.fit(term_docs_train, y_train)
test_pred = NB.predict(test_term_docs)

label_index {0: defaultdict(<class 'list'>, {-1: [0, 1, 2, 3, 4, 5, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 194, 195, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 

lab [[ 672    0    1 ...    0    0    9]
 [ 679    0    0 ...    0    1    4]
 [ 164    0    0 ...    0    0    0]
 ...
 [ 427    0    2 ...    0    0    0]
 [ 116    0    0 ...    0    0    0]
 [1144    0   16 ...    0    0    2]] [[ 462    0    0 ...    0    0    1]
 [ 179    0    0 ...    0    0    0]
 [ 329    0    0 ...    0    0    1]
 ...
 [1718    0    5 ...    0    0    7]
 [ 536    0    1 ...    1    0    0]
 [ 118    0    0 ...    0    0    0]]
sum (1000,) [22182     5    43     3     2    15    19     5    14    14     5    19
     9     5    55     9     3     6     7    13    15     6    17    14
    14    12    10    11    14    16     8     9     9    10    29    14
     2    14     4     5    19     3    34    62     5    16    13    21
    15     3     4     4     5    50     8     8     0    63     6     5
    15     3     6     6     5    12    13   157     6     3    58     2
    51    12     2     5    84     4    16    13     8    28     3     7
     5     8     

lab [[ 672    0    1 ...    0    0    9]
 [ 679    0    0 ...    0    1    4]
 [ 164    0    0 ...    0    0    0]
 ...
 [ 233    0    1 ...    0    0    1]
 [ 427    0    2 ...    0    0    0]
 [1144    0   16 ...    0    0    2]] [[1332    3    2 ...    0    3    9]
 [1803    1    3 ...    0    2   22]
 [ 184    1    0 ...    0    0    1]
 ...
 [ 582    0    0 ...    0    0    0]
 [  14    0    0 ...    0    0    0]
 [ 116    0    0 ...    0    0    0]]
sum (1000,) [31466    17    95    15     9    19    16     4    15    15     9    28
    16     6    52    18     3     7     4    44    10     7    12     9
    18     7    18     3    17    20     9    14     9    23    35     9
     4     6     6    17    16    10    42    72    13     9     5    17
    14     4     8    11     7    48    10     8    11    76     7    11
    11     5     5    12    13    13    22   288     4     6    97     8
    99     9     9     7   154     5    44     3     8    59     5    15
     5    21    1

In [207]:
test_pred

array([4, 3, 4, 1, 2, 3, 3, 3, 3, 4, 3, 3, 0, 0, 4, 4, 4, 3, 4, 4, 1, 4,
       0, 4, 3, 4, 4, 4, 4, 4, 4, 1, 0, 4, 4, 3, 4, 0, 0, 4, 4, 4, 4, 2,
       4, 4, 4, 4, 1, 3, 3, 4, 4, 3, 0, 4, 1, 3, 3, 4, 4, 4, 2, 4, 3, 1,
       4, 4, 3, 4, 0, 3, 4, 4, 4, 4, 4, 4, 3, 4, 3, 1, 4, 4, 4, 1, 3, 4,
       4, 1, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 1, 3, 3, 2, 0,
       4, 4, 4, 0, 3, 3, 4, 1, 4, 1, 3, 1, 2, 2, 4, 3, 3, 4, 4, 2, 4, 4,
       4, 0, 3, 4, 0, 1, 3, 4, 1, 4, 4, 3, 4, 3, 4, 4, 1, 3])

In [209]:
pd.DataFrame(test_pred).to_csv("2018310412_2nd.csv", index=False)